In [1]:
from bs4 import BeautifulSoup, Tag, NavigableString
from collections import defaultdict
import re
from itertools import islice
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from googletrans import Translator

In [2]:
whole_dict = {
    'protected': {
        'df':
        pd.read_csv(f'data/data_files/pipeline_steps/protected_articles/7.1_maskadded.csv'),
        'titles':
        pd.read_csv(f'data/data_files/pipeline_steps/protected_articles/0_protected_titles.csv')
    },
    'excellent': {
        'df':
        pd.read_csv(f'data/data_files/pipeline_steps/excellent_articles/7.1_maskadded.csv'),
        'titles':
        pd.read_csv(f'data/data_files/pipeline_steps/excellent_articles/0_excellent_titles.csv')
    },
    'readworthy': {
        'df':
        pd.read_csv(
            f'data/data_files/pipeline_steps/readworthy_articles/7.1_maskadded.csv'),
        'titles':
        pd.read_csv(f'data/data_files/pipeline_steps/readworthy_articles/0_readworthy_titles.csv')
    },
}

In [9]:
def unique_words(sentences):
    # Join all sentences into one string
    joined_text = ' '.join(sentences)

    # Split the string into words
    words = joined_text.split()

    # Use a set to find unique words
    unique_words = set(words)

    # Print unique words
    for word in unique_words:
        print(word)

In [15]:
b[0][0]

'['

In [14]:
unique_words(b[0])

D
,
R
B
M
]
C
P
A
E
V
T
'
[
U
N
X
J
S
O


In [7]:
b=a['pos_resolved_sentence_list'].iloc[0:100]

In [3]:
whole_dict['protected']['df'].columns


Index(['Unnamed: 0', 'article_index', 'nopos_resolved_text',
       'nopos_nonresolved_text', 'pos_nonresolved_text', 'pos_resolved_text',
       'quot_label', 'link_label', 'namelink_label',
       'pos_nonresolved_quot_filter', 'pos_nonresolved_link_filter',
       'pos_nonresolved_namelink_filter', 'pos_resolved_quot_filter',
       'pos_resolved_link_filter', 'pos_resolved_namelink_filter',
       'masked_nonresolved_text', 'masked_resolved_text'],
      dtype='object')

In [ ]:
whole_dict['protected']['df'][whole_dict['protected']['df']['quot_label']==True]

In [6]:
whole_dict['protected']['df'].columns

Index(['Unnamed: 0', 'article_index', 'nopos_resolved_text',
       'nopos_nonresolved_text', 'pos_nonresolved_text', 'pos_resolved_text',
       'quot_label', 'link_label', 'namelink_label',
       'pos_nonresolved_quot_filter', 'pos_nonresolved_link_filter',
       'pos_nonresolved_namelink_filter', 'pos_resolved_quot_filter',
       'pos_resolved_link_filter', 'pos_resolved_namelink_filter',
       'masked_nonresolved_text', 'masked_resolved_text'],
      dtype='object')

In [7]:
df = whole_dict['protected']['df']

In [8]:
# defining the corresponding column pairs
column_pairs = [("quot_label", "pos_nonresolved_quot_filter"),
                ("link_label", "pos_nonresolved_link_filter"),
                ("namelink_label", "pos_nonresolved_namelink_filter")]

# initializing a dictionary to store the results
results = {}

# iterating over the column pairs
for col_a, col_b in column_pairs:
    # count the number of rows where col_a is False and col_b is True
    count = df[(df[col_a] == False) & (df[col_b] == True)].shape[0]
    results[(col_a, col_b)] = count

# print the results
for key, value in results.items():
    print(f"For pair {key}, the count is {value}")


For pair ('quot_label', 'pos_nonresolved_quot_filter'), the count is 20971
For pair ('link_label', 'pos_nonresolved_link_filter'), the count is 19235
For pair ('namelink_label', 'pos_nonresolved_namelink_filter'), the count is 16463


In [10]:
import pandas as pd


# defining the corresponding column pairs
column_pairs = [("quot_label", "pos_nonresolved_quot_filter"),
                ("link_label", "pos_nonresolved_link_filter"),
                ("namelink_label", "pos_nonresolved_namelink_filter")]

# initializing a list to store the results
results = []

# iterating over the column pairs
for col_a, col_b in column_pairs:
    # count the number of rows where col_a is False and col_b is True
    count_true = df[(df[col_a] == False) & (df[col_b] == True)].shape[0]
    # count the number of rows where either col_a is not False or col_b is not True
    count_false = df.shape[0] - count_true
    # append the results to the list
    results.append([col_a, df.shape[0], count_true, count_false])

# create a DataFrame from the results
results_df = pd.DataFrame(results,
                          columns=["col_a", "total_size", "condition_true", "condition_false"])

# print the results DataFrame
print()
print(results_df)


            col_a  total_size  condition_true  condition_false
0      quot_label      289843           20971           268872
1      link_label      289843           19235           270608
2  namelink_label      289843           16463           273380


In [16]:
for key, value in whole_dict.items():
    df = value['df']
    # defining the corresponding column pairs
    column_pairs = [("quot_label", "pos_nonresolved_quot_filter"),
                    ("link_label", "pos_nonresolved_link_filter"),
                    ("namelink_label", "pos_nonresolved_namelink_filter")]

    # initializing a list to store the results
    results = []

    # iterating over the column pairs
    for col_a, col_b in column_pairs:
        # count the number of rows where col_a is False and col_b is True
        count_true = df[(df[col_a] == False) & (df[col_b] == True)].shape[0]
        # count the number of rows where either col_a is not False or col_b is not True
        count_false = df.shape[0] - count_true
        # append the results to the list
        share_true = df[col_a].mean()
        # drop rows that match the condition and calculate the share of True values in the remaining rows
        count_a_true_b_true = df[(df[col_a] == True) & (df[col_b] == True)].shape[0]
        share_true_after_drop = df.drop(df[(df[col_a] == False) & (df[col_b] == True)].index)[col_a].mean()
        unique_pos_nonresolved_text = df[(df[col_a] == True)
                                         & (df[col_b] == True)]['pos_nonresolved_text'].nunique()
        results.append([
            col_a, df.shape[0], count_true, count_false, (count_true / df.shape[0]), share_true,
            share_true_after_drop, count_a_true_b_true, unique_pos_nonresolved_text
        ])

    # create a DataFrame from the results
    results_df = pd.DataFrame(results,
                            columns=["Label", "Total sentences", "Filtered sentences", "Remaining sentences",'Share filtered','Postive Rate before','Positive rate after','No. matching POS-seqs','Unique POS'])

    # print the results DataFrame
    print(key)
    display(results_df)

protected


,Label,Total sentences,Filtered sentences,Remaining sentences,Share filtered,Postive Rate before,Positive rate after,No. matching POS-seqs,Unique POS
0,quot_label,289843,20971,268872,0.072353,0.251412,0.271021,7289,3924
1,link_label,289843,19235,270608,0.066364,0.622730,0.666994,16064,5829
2,namelink_label,289843,16463,273380,0.056800,0.741432,0.786082,18374,5679


excellent


,Label,Total sentences,Filtered sentences,Remaining sentences,Share filtered,Postive Rate before,Positive rate after,No. matching POS-seqs,Unique POS
0,quot_label,590756,58069,532687,0.098296,0.234428,0.259984,17404,9567
1,link_label,590756,54588,536168,0.092404,0.553379,0.609719,36996,14253
2,namelink_label,590756,45896,544860,0.077690,0.704541,0.763888,45699,14149


readworthy


,Label,Total sentences,Filtered sentences,Remaining sentences,Share filtered,Postive Rate before,Positive rate after,No. matching POS-seqs,Unique POS
0,quot_label,651668,66078,585590,0.101398,0.214348,0.238535,18439,10145
1,link_label,651668,62417,589251,0.095780,0.563186,0.622842,42822,15833
2,namelink_label,651668,52707,598961,0.080880,0.702017,0.763793,52078,15582
